# Глава 6. Интерлюдия о динамической типизации

## Отсутствие инструкций объявления

**Динамическая типизация** - типы данных определяются автоматически во время выполнения, а не в результате объявлений в программном коде.

### Переменные, объекты и ссылки

**Создание переменной** - переменная создается тогда, когда в коде ей присваивается некоторое значение. Все последующие операции присваивания просто изменяют значение

**Типы переменных** - переменные не имеют никакой информации о типе или ограничениях, связанных с ним. Понятие типа присуще объектам, а не именам. Переменные универсальны по своей природе - они являются всего лишь **ссылками на конкретные объекты в конкретные моменты времени**

**Использование переменной** - когда переменная участвует в выражениях, ее имя замещается объектом, на который она в настоящий момент ссылается, независимо от того, что это за объект

> В итоге складывается следующая картина: переменные создаются при выполнении операции присваивания, могут ссылаться на объекты любых типов и им должны быть присвоены некоторые значения, прежде чем к ним можно будет обратиться.

_Пример_

`a = 3`

1. Создается **объект**, представляющий число 3
2. Создается **переменная** а, если она еще не существует
3. В переменную a записывается **ссылка** на вновь созданный объект, представляющий число 3

> Переменные и объекты хранятся в разных частях памяти и связаны между собой

> **Переменные всегда ссылаются на объекты и никогда - на другие переменные**, но крупные объекты могут ссылаться на другие объекты

**Переменные** – это записи в системной таблице, где предусмотрено место для хранения ссылок на объекты.

**Объекты** – это области памяти с объемом, достаточным для  представления значений этих объектов.

**Ссылки** – это автоматически разыменовываемые указатели на объекты.

С технической точки зрения объекты имеют более сложную структуру, чем
просто пространство в памяти, необходимое для хранения значения.

>Каждый объект имеет два стандартных поля:
>- **описатель типа**, используемый для хранения информации о типе объекта, и
>- **счетчик ссылок**, используемый для определения момента, когда память, занимаемая объектом, может быть освобождена.

### Информация о типе хранится в объекте, но не в переменной

Имена не имеют типов. А объекты знают, к какому типу относятся - каждый объект содержит поле, в котором хранится информация о его типе.

### Объекты уничтожаются автоматически

**Сборка мусора (garbage collection)** - автоматическое освобождение памяти, занимаемой объектами, когда на него не ссылается какое-либо другое имя или объект

Достигается это за счет того, что в каждом объекте имеется **счетчик ссылок**, указывающих на объект в настоящий момент времени. Как только (и именно в этот момент) значение счетчика достигнет нуля, память, занимаемая объектом, автоматически освобождается.

**Циклические ссылки** - существует возможность создать в объекте ссылку на самого себя или на другой объект, который ссылается сам на себя.

### Разделяемые ссылки

```
a = 3
b = a
```
Вторая инструкция вынуждает интерпретатор создать переменную `b` и использовать для инициализации переменную `a`, при этом она замещается объектом, на который ссылается (`3`), и `b` превращается в ссылку на этот объект. В результате переменные `a` и `b` ссылаются на один и тот же объект (указывают на одну и ту же область в памяти). Т.е это **разделяемая ссылка** - несколько имен ссылаются на один и тот же объект.

Если дальше присвоить `a = 'spam'`, то `b` продолжит ссылаться на `3`.

### Разделяемые ссылки и изменяемые объекты

Существуют такие объекты и операции, которые приводят к изменению самих объектов. При работе с такими объектами необходимо быть особенно внимательным при использовании разделяемых ссылок, т.к. изменение одного имени может отразиться на других именах.

```
>>> L1 = [2, 3, 4]
>>> L2 = L1
>>> L1[0] = 24

>>> L1
[24, 3, 4]

>>> L2
[24, 3, 4]
```

Здесь мы не изменяем сам объект L1, изменяется компонент объекта, на который ссылается L1.

> **Копирование списка**: `L2 = L1[:]`

Для словарей и не только имеется `copy`

```
import copy

# Создание поверхностной копии любого объекта Y
X = copy.copy(Y)

# Создание полной копии - копируются все вложенные части
X = copy.deepcopy(Y)
```

### Разделяемые ссылки и равенство

```
x = 42
x = 'shrubbery'
```

Объект `42` теперь уничтожен?

Т.к. интерпретатор Python кэширует и повторно использует малые целые числа и небольше строки, объект 42 скорее всего не будет уничтожен. Он, вероятнее всего останется в системной таблице для повторного использования, когда вы вновь сгенерируете число 42 в программном коде.

Однако большинство объектов уничтожаются немедленно, как только будет потеряна последняя ссылка.

**==** - проверяет, равны ли значения объектов

**is** - проверяет идентичность объектов, возвращает `True`, только если оба имени ссылаются на один и тот же объект.

На самом деле оператор `is` просто сравнивает указатели, которые реализуют ссылки, и тем самым может использоваться для выявления разделяемых ссылок в коде.

```
>>> L = [1, 2, 3]
>>> M = [1, 2, 3]  # M и L ссылаются на разные объекты
>>> L == M  # Одно и то же значение
True
>>> L is M  # Но разные объекты
False
```

```
>>> X = 42
>>> Y = 42 # Должно получиться два разных объекта
>>> X == Y
True
>>> X is Y # Тот же самый объект: кэширование в действии!
True
```

> Функция **`getrefcount`** из стандартного модуля **`sys`** возвращает значение поля счетчика ссылок в объекте